In [15]:
import os, datetime
import cv2
import xml.etree.ElementTree as ET
from PIL import Image
import numpy as np
import random
from pathlib import Path

pre_fix = '/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_zhuhai_add/VOC2007/'
generated_xml_dir = pre_fix + '/Annotations_/'
generated_image_dir = pre_fix + '/JPEGImages_/'

image_dir = pre_fix + '/seed_i/'
xml_dir = pre_fix + '/seed_x/'

'''
time_str = (datetime.datetime.now()).strftime("%Y%m%d")
pre_fix = '/media/yyp/DATA/'
generated_xml_dir = pre_fix + time_str + '_xml_replaced_bkg/'
generated_image_dir = pre_fix + time_str + '_image_replaced_bkg/'

image_dir = pre_fix + '20190922_image/'
'''

'\ntime_str = (datetime.datetime.now()).strftime("%Y%m%d")\npre_fix = \'/media/yyp/DATA/\'\ngenerated_xml_dir = pre_fix + time_str + \'_xml_replaced_bkg/\'\ngenerated_image_dir = pre_fix + time_str + \'_image_replaced_bkg/\'\n\nimage_dir = pre_fix + \'20190922_image/\'\n'

In [16]:
#os.system('mkdir -p ' + xml_dir)
#os.system('mkdir -p ' + image_dir)
os.system('mkdir -p ' + generated_image_dir)
os.system('mkdir -p ' + generated_xml_dir)

SEED_XML_DIR = xml_dir
SEED_IMG_DIR = image_dir
GENE_IMG_DIR = generated_image_dir
GENE_XML_DIR = generated_xml_dir
SEED_BKG_DIR = pre_fix + 'bg/'

def past_to_background_from_image_file(file, bboxes, background_img_array, tree, extend_spaces=0):
    
    img = cv2.imread(file)
    #img = img - 50
    '''if(img.shape != background_img_array.shape):
        print('shape not match')
        return'''
    #print(img.shape)
    #print(img)
    img_objs = []
    for bbox in bboxes:
        img_obj = img[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])]
        img_objs.append(img_obj)
    i = 0
    for bbox in bboxes:
        background_img_array[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])] = img_objs[i]
        i = i+1
    time_mark = datetime.datetime.now()
    time_str = time_mark.strftime("%Y%m%d%H%M%S%f_")
    cv2.imwrite(GENE_IMG_DIR + time_str + file.split('/')[-1], background_img_array)
    xml_name = (file.split('/')[-1])[:-3] + 'xml'
    tree.write(generated_xml_dir + time_str + xml_name)
    return 
def get_obj_from_xml(xml):
    in_file = open(xml_name)
    tree=ET.parse(in_file)
    root = tree.getroot()
    return [obj for obj in root.iter('object')]
def get_bboxes_from_etree(etree):
    root = tree.getroot()  
    objects = root.findall('object')
    bboxes = []
    for obj in objects:
        '''
        difficult = obj.find('difficult').text
        cls_ = obj.find('name').text
        if cls_ not in classes_ or int(difficult)==1:
            continue
        '''
        xmlbox = obj.find('bndbox')
        b = [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]
        bboxes.append(b)
    return bboxes
def generate_new_xmlobj(xmlobj_old, new_position, new_size):
    element_object = ET.Element('object')
    tag_name = ET.SubElement(element_object, 'name')
    tag_name.text = xmlobj_old.find('name').text

    tag_difficult = ET.SubElement(element_object, 'difficult')
    tag_difficult.text = xmlobj_old.find('difficult').text

    element_bndbox = ET.SubElement(element_object, 'bndbox')
    tag_xmin = ET.SubElement(element_bndbox, 'xmin')
    tag_ymin = ET.SubElement(element_bndbox, 'ymin')
    tag_xmax = ET.SubElement(element_bndbox, 'xmax')
    tag_ymax = ET.SubElement(element_bndbox, 'ymax')
    tag_xmin.text = str(new_position[0])
    tag_ymin.text = str(new_position[1])
    tag_xmax.text = str(new_position[0] + new_size[1])
    tag_ymax.text = str(new_position[1] + new_size[0])
    return element_object
    
def past_and_insert(img_obj, img_array, new_position, obj_element, etree):
    new_xmlobj = generate_new_xmlobj(obj_element, new_position, img_obj.shape[:2])
    new_xml_etree = insert_to_xml(new_xmlobj, etree)
    new_pil_img = past_obj_to_background(img_obj, img_array, new_position)
    return new_pil_img, new_xml_etree
    

def insert_to_xml(xml_obj, xml_etree):
    root = xml_etree.getroot()
    root.append(xml_obj)
    return xml_etree
    
def past_obj_to_background(img_obj, img_array, position = (200, 200)):
    img = Image.fromarray(img_array)
    img_obj = Image.fromarray(img_obj)
    img.paste(img_obj, position)
    return img
    
def get_cls_from_xmlobj(obj_element):    
    return obj_element.find('name').text

def get_bbox_from_xmlobj(obj_element):
    xmlbox = obj_element.find('bndbox')
    return [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), 
            float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]
def generate_new_position(img_size, img_obj_size):
    array_x = np.arange(int(img_size[1] - img_obj_size[1]))
    array_y = np.arange(int(img_size[0] - img_obj_size[0]))
    random_x = random.sample(list(array_x), 1)[0]
    random_y = random.sample(list(array_y), 1)[0]
    new_position = (random_x, random_y)
    #print(new_position)
    return new_position
def generate_new_bbox(img_size, img_obj_size):
    array_x = np.arange(int(img_size[1] - img_obj_size[1]))
    array_y = np.arange(int(img_size[0] - img_obj_size[0]))
    random_x = random.sample(list(array_x), 1)[0]
    random_y = random.sample(list(array_y), 1)[0]
    new_position = (random_x, random_y)
    new_bbox = [random_x, random_x + img_obj_size[1], random_y, random_y + img_obj_size[0]]
    #print(new_bbox)
    return new_bbox
    
    
def inset_obj_to_an_image_and_xml(img_obj, img, obj_element, etree):
    new_bbox = generate_new_bbox(img.shape[:2], img_obj.shape[:2])
    bboxes = get_bboxes_from_etree(etree)
    while(not check_bbox(new_bbox, bboxes)):
        print('new_bbox not suitable, retry...')
        new_bbox = generate_new_bbox(img.shape[:2], img_obj.shape[:2])
    print('new_bbox succussful')
    new_pil_img, new_xml_etree = past_and_insert(img_obj, img, (new_bbox[0], new_bbox[2]), obj_element, tree)
    
    return new_pil_img, new_xml_etree

def check_bbox(new_bbox, bboxes):
    for bbox in bboxes:
        if(IOU(new_bbox, bbox) > 0.005):
            return False
        continue
    return True

seed_xml_names = os.listdir(SEED_XML_DIR)
seed_bkg_names = os.listdir(SEED_BKG_DIR)
seed_bkg_names.sort()
seed_xml_names.sort()

for xml_name in seed_xml_names:
    '''xml_name_part = xml_name[-15:-4]
    print(xml_name_part)
    if not (int(xml_name_part) >= 460000):
        continue'''
    #print(xml_name)
    FOUND_FLAG = False
    for i in seed_bkg_names:
        #if(i[:9]!= xml_name[:9]):
        #    continue
        #print("find")
        FOUND_FLAG = True
        background_img_data = cv2.imread(SEED_BKG_DIR + i)
        #print(background_img_data.shape)
    
        if(not xml_name.endswith('.xml')):
            continue
        in_file = open(SEED_XML_DIR + xml_name)
        tree=ET.parse(in_file)
        bboxes = get_bboxes_from_etree(tree)
        #print(SEED_IMG_DIR + xml_name[:-3] + 'jpg')
        past_to_background_from_image_file(SEED_IMG_DIR + xml_name[:-3] + 'jpg', bboxes, background_img_data, tree)
    if(FOUND_FLAG==False):
        print('not found' + xml_name)